In [1]:
import sys
sys.path.append("..") # Adds the module to path

# DeepTrack.features

This notebook introduces the DeepTrack.features module.

## What is a feature?

Features are instances of the abstract class `Feature` and are responsible for modifying images. Each implementation of the class `Feature` needs to define a method `get(image, **kwargs)`, takes an image as an input and should return an updated image. This additive, such as adding a particle or gaussian noise, or non-additive, such as poisson noise or a low-pass filter. The image that is modified is in itself an instance of the class Image, which for now can be thought of as normal numpy ndarrays.

It is very important to note that features do _not_ create images, they modify images. As such, they always need an image as an input. 



## Operations defind on features

Features can be combined to create a (TODO: better word for feature tree), which creates images composed of the combined features. This is done using the add operator (+), the multiply operator (\*), and the power operator (\*\*). 

The Add operator combines two features such that the output of the left side is set as the input to the feature to the right. For example, `foobar = foo + bar`, where foo and bar are features. Here, when an image is resolved from foobar, it first resolves an image from foo, then uses this image as the input to resolve an image from bar.



*

In [2]:
from DeepTrack.features import Feature
from DeepTrack.Image import Image

class Foo(Feature):
    def get(image,**kwargs):
        print("Calling foo")
        return image
    
class Bar(Feature):
    def get(image,**kwargs):
        print("Calling foo")
        return image
    
foobar = foo() + bar()

foobar.resolve(Image([]))

ModuleNotFoundError: No module named 'DeepTrack.Distributions'